# Streamer Revisit Prediction on Twitch Data

In this part we predict whether a user will revisit the same streamer in the near future.  
The dataset consists of Twitch chat connections sampled every 10 minutes over 43 days.

Each row has:
- user_id  
- stream_id  
- streamer  
- time_start  
- time_stop  

Durations are measured in 10-minute steps.

Basic dataset characteristics:
- 100k users  
- 162.6k streamers  
- ~3M interactions  
- 6148 total time steps  

We formulate revisit prediction as a binary classification and ranking task.  
For each interaction, we predict whether the user will return to the same streamer within a fixed time window.  
This aligns with temporal recommender-style modeling and session behavior prediction.

## Imports and constants

In this part we import required libraries and define constants that control the experiment setup.  
We separate constants because they govern the behavior of the model and allow consistent reproducibility.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve
from sklearn.calibration import calibration_curve

from IPython.display import display
import pickle

In [ ]:
# Ensures reproducible random behavior throughout the notebook
np.random.seed(77)

# Eensures reproducible training
RANDOM_STATE = 77

# Defines the look-ahead window:
# a user counts as a revisit if they return to the same streamer within this many time steps
PREDICTION_WINDOW_STEPS = 12

# Determines how much of the data is used as the test set (time based here)
TEST_FRACTION = 0.2

# Controls ranking evaluation:
# precision@10 and recall@10 are computed using the top 10 predicted streamers per user
TOP_K = 10

## Load data and basic preprocessing

In this part we load the raw Twitch dataset and prepare it for modeling.

We:
- assign column names,
- compute duration as `time_stop - time_start`,
- clip it to at least one time step so all durations are positive,
- sort interactions per user by time to enforce temporal order.

We do this because all later feature engineering and label creation depend on chronological user histories.

In [ ]:
df = pd.read_csv("100k_a.csv", header=None)
df.columns = ["user_id", "stream_id", "streamer", "time_start", "time_stop"]

df["duration_steps"] = (df["time_stop"] - df["time_start"]).clip(lower=1)

df = df.sort_values(["user_id", "time_start"]).reset_index(drop=True)
df.head()

## Label construction: revisit within a fixed time window

In this part we create the binary target label.  
For each user–streamer pair, we look at the next interaction with the same streamer and measure the time gap.

If the next interaction occurs within `PREDICTION_WINDOW_STEPS`, we assign label = 1.  
Otherwise label = 0.

We do this because the modeling objective is to identify near-future revisits rather than long-term interest.

In [ ]:
df = df.sort_values(["user_id", "streamer", "time_start"]).reset_index(drop=True)

next_start = df.groupby(["user_id", "streamer"])["time_start"].shift(-1)
delta = next_start - df["time_start"]
df["label"] = ((delta <= PREDICTION_WINDOW_STEPS) & delta.notna()).astype(int)

df = df.sort_values(["time_start", "user_id"]).reset_index(drop=True)
df.head()

## Feature engineering

In this part we engineer features capturing user–streamer history, overall user behavior, streamer characteristics, similarity signals, and temporal dynamics.

We do this because raw IDs cannot be used directly by logistic regression, and these behavioral features encode meaningful signals relevant to revisit likelihood.

### User–streamer historical features

These features describe how much the user has previously interacted with this streamer.

We compute:
- frequency of past interactions,
- cumulative duration with the streamer,
- average duration per interaction,
- recency since last interaction.

We do this because stronger and more recent user–streamer relationships increase revisit probability.

In [ ]:
df = df.sort_values(["user_id", "streamer", "time_start"]).reset_index(drop=True)

df["freq_user_streamer"] = df.groupby(["user_id", "streamer"]).cumcount()

cumdur = df.groupby(["user_id", "streamer"])["duration_steps"].cumsum()
df["totaldur_user_streamer"] = cumdur - df["duration_steps"]

df["avgdur_user_streamer"] = 0.0
mask = df["freq_user_streamer"] > 0
df.loc[mask, "avgdur_user_streamer"] = df.loc[mask, "totaldur_user_streamer"] / df.loc[mask, "freq_user_streamer"]

prev_start = df.groupby(["user_id", "streamer"])["time_start"].shift(1)
df["recency_user_streamer"] = df["time_start"] - prev_start

df["recency_user_streamer"] = df["recency_user_streamer"].fillna(df["recency_user_streamer"].max())

df.head()

### Global temporal features

These features describe activity patterns independent of specific streamers.

We compute:
- `recency_global_user`: time since the user's previous interaction with any streamer,
- `recency_global_streamer`: time since the streamer’s previous interaction from any user.

We do this because user activity rhythms and streamer traffic levels influence revisit likelihood.

In [ ]:
df = df.sort_values(["user_id", "time_start"])
prev_u = df.groupby("user_id")["time_start"].shift(1)
df["recency_global_user"] = (df["time_start"] - prev_u).fillna((df["time_start"] - prev_u).max())

df = df.sort_values(["streamer", "time_start"])
prev_s = df.groupby("streamer")["time_start"].shift(1)
df["recency_global_streamer"] = (df["time_start"] - prev_s).fillna((df["time_start"] - prev_s).max())

df = df.sort_values(["user_id", "time_start"]).reset_index(drop=True)
df.head()

### User breadth and session index

Here we capture the user’s exploration behavior and their position within their interaction timeline.

We compute:
- number of distinct streamers so far,
- session index,
- log-transformed session index.

We do this because users who explore widely behave differently from users who stay focused, and early vs. late session behaviors differ.

In [ ]:
def cum_nunique(vals):
    s = set()
    out = []
    for v in vals:
        out.append(len(s))
        s.add(v)
    return pd.Series(out, index=vals.index)

df["num_distinct_streamers_user"] = df.groupby("user_id")["streamer"].transform(cum_nunique)
df["session_idx"] = df.groupby("user_id").cumcount()
df["log_session_idx"] = np.log1p(df["session_idx"])
df.head()

### Streamer global statistics

These describe long-term streamer popularity patterns.

We compute:
- event count,
- number of unique users,
- total duration,
- average duration.

We do this because popular streamers and niche streamers attract very different revisit dynamics.

In [ ]:
streamer_counts = df.groupby("streamer")["user_id"].count().rename("streamer_event_count")
streamer_users = df.groupby("streamer")["user_id"].nunique().rename("streamer_unique_users")
streamer_dur = df.groupby("streamer")["duration_steps"].sum().rename("streamer_total_duration")

stats = pd.concat([streamer_counts, streamer_users, streamer_dur], axis=1)
stats["streamer_avg_duration"] = stats["streamer_total_duration"] / stats["streamer_event_count"]

df = df.merge(stats, on="streamer", how="left")
df.head()

### Item–item co-visitation similarity

Here we compute similarity between streamers by counting how often the same user watches both.

We do this because collaborative filtering signals help capture user taste patterns not visible from single-streamer history alone.

In [ ]:
items_per_user = df.groupby("user_id")["streamer"].apply(lambda x: sorted(x.unique()))
item_freq = df.groupby("streamer")["user_id"].nunique()

pair_count = defaultdict(int)

for items in items_per_user:
    n = len(items)
    for i in range(n):
        for j in range(i + 1, n):
            a, b = items[i], items[j]
            pair_count[(a, b)] += 1
            pair_count[(b, a)] += 1

item_sim = { (a, b): c / np.sqrt(item_freq[a] * item_freq[b]) for (a, b), c in pair_count.items() }

### Item similarity feature

For each interaction, we compute the average similarity between the current streamer and all streamers the user has watched before.

We do this because users tend to revisit streamers similar to their previous preferences.

In [ ]:
df = df.sort_values(["user_id", "time_start"])
df["item_sim_mean"] = 0.0

user_hist = defaultdict(set)

for idx, row in df.iterrows():
    u = row.user_id
    s = row.streamer
    others = user_hist[u] - {s}
    if others:
        sims = [item_sim.get((s, o), 0.0) for o in others]
        df.at[idx, "item_sim_mean"] = float(np.mean(sims))
    user_hist[u].add(s)

### Share-of-duration feature

This feature describes how much of the user’s historical watch time belonged to this streamer.

We do this because a high share indicates strong preference.

In [ ]:
df = df.sort_values(["user_id", "time_start"]).reset_index(drop=True)

user_cum = df.groupby("user_id")["duration_steps"].cumsum()
df["user_total_before"] = user_cum - df["duration_steps"]

df["share_dur_streamer_in_user"] = 0.0
mask = df["user_total_before"] > 0
df.loc[mask, "share_dur_streamer_in_user"] = df.loc[mask, "totaldur_user_streamer"] / df.loc[mask, "user_total_before"]

df["share_dur_streamer_in_user"] = df["share_dur_streamer_in_user"].fillna(0.0)
df = df.drop(columns=["user_total_before"])
df.head()

### Log-transform skewed features

We apply log transforms to several highly skewed count/duration features to stabilize scale for logistic regression.

In [ ]:
skew_cols = [
    "freq_user_streamer",
    "totaldur_user_streamer",
    "recency_user_streamer",
    "recency_global_user",
    "recency_global_streamer",
    "streamer_event_count",
    "streamer_unique_users",
    "streamer_total_duration"
]

for c in skew_cols:
    df[f"log_{c}"] = np.log1p(df[c])

## Train–test split

We sort by time and assign the earliest part of the dataset to training and the most recent part to testing.

We do this because temporal prediction must not leak future information into training.

In [ ]:
df = df.sort_values("time_start").reset_index(drop=True)
n = len(df)
n_test = int(TEST_FRACTION * n)

train_df = df.iloc[:-n_test].copy()
test_df = df.iloc[-n_test:].copy()

y_train = train_df["label"].values
y_test = test_df["label"].values

## Feature matrix and scaling

We standardize features because logistic regression performs better when inputs share a common scale.

In [ ]:
feature_cols = [
    "log_freq_user_streamer",
    "log_totaldur_user_streamer",
    "avgdur_user_streamer",
    "log_recency_user_streamer",
    "share_dur_streamer_in_user",
    "num_distinct_streamers_user",
    "log_streamer_event_count",
    "log_streamer_unique_users",
    "log_streamer_total_duration",
    "streamer_avg_duration",
    "log_recency_global_user",
    "log_recency_global_streamer",
    "item_sim_mean",
    "log_session_idx"
]

scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[feature_cols])
X_test = scaler.transform(test_df[feature_cols])

In [ ]:
safe_cols = [
    "freq_user_streamer",
    "totaldur_user_streamer",
    "recency_user_streamer",
    "recency_global_user",
    "recency_global_streamer",
    "streamer_event_count",
    "streamer_unique_users",
    "streamer_total_duration"
]

for c in safe_cols:
    train_df[c] = train_df[c].clip(lower=0)
    test_df[c] = test_df[c].clip(lower=0)

train_df[feature_cols] = train_df[feature_cols].fillna(0)
test_df[feature_cols] = test_df[feature_cols].fillna(0)

print("Train NaNs:", train_df[feature_cols].isna().sum().sum())
print("Test NaNs:", test_df[feature_cols].isna().sum().sum())

## Baseline models

We include trivial baselines to contextualize the performance of learned models.

In [ ]:
random_scores = np.random.rand(len(test_df))

streamer_pop_train = train_df["streamer"].value_counts()
pop_scores = test_df["streamer"].map(streamer_pop_train).fillna(0).values

user_streamer_counts = train_df.groupby(["user_id", "streamer"]).size()

def user_top_score(row):
    return user_streamer_counts.get((row.user_id, row.streamer), 0)

user_top_scores = test_df.apply(user_top_score, axis=1).values

## Evaluation helpers

We compute both threshold-based evaluation metrics and ranking metrics such as precision@K and recall@K.

In [ ]:
def evaluate_basic(y_true, scores, th=0.5):
    preds = (scores >= th).astype(int)
    return {
        "accuracy": accuracy_score(y_true, preds),
        "roc_auc": roc_auc_score(y_true, scores),
        "precision": precision_score(y_true, preds, zero_division=0),
        "recall": recall_score(y_true, preds, zero_division=0),
        "f1": f1_score(y_true, preds, zero_division=0)
    }

def evaluate_at_k(test_df, scores, k=10):
    df2 = test_df.copy()
    df2["score"] = scores
    prec = []
    rec = []
    for u, g in df2.groupby("user_id"):
        g = g.sort_values("score", ascending=False)
        topk = g.head(k)
        hits = topk["label"].sum()
        total = g["label"].sum()
        if total > 0:
            prec.append(hits / k)
            rec.append(hits / total)
    return {f"precision@{k}": np.nanmean(prec), f"recall@{k}": np.nanmean(rec)}

## Logistic Regression with hyperparameter tuning

We tune over penalty types and regularization strengths to find the model maximizing ROC AUC.

We do this because logistic regression is sensitive to its regularization settings.

In [ ]:
C_values = [0.01, 0.1, 1.0, 3.0, 10.0]
penalties = ["l1", "l2"]

best_lr_auc = -1
best_lr_model = None
best_lr_params = None

for C in C_values:
    for pen in penalties:
        try:
            model = LogisticRegression(
                max_iter=4000,
                class_weight="balanced",
                solver="saga",
                penalty=pen,
                C=C,
                random_state=RANDOM_STATE
            )

            model.fit(X_train, y_train)

            preds = model.predict_proba(X_test)[:, 1]

            auc = roc_auc_score(y_test, preds)

            if auc > best_lr_auc:
                best_lr_auc = auc
                best_lr_model = model
                best_lr_params = (C, pen)

        except Exception as e:
            print(f"Failed for C={C}, penalty={pen}: {e}")

# Safety check
if best_lr_model is None:
    raise RuntimeError("No logistic regression model successfully trained.")

# Final best model
logreg = best_lr_model
logit_scores = logreg.predict_proba(X_test)[:, 1]


KeyboardInterrupt: 

## Collaborative filtering scoring

We compute CF scores by averaging similarity to all streamers a user has watched before.

We do this because CF captures shared user taste patterns absent from explicit features.

In [ ]:
cf_scores = []
user_history_train = defaultdict(list)

for _, row in train_df.iterrows():
    user_history_train[row.user_id].append(row.streamer)

for _, row in test_df.iterrows():
    u = row.user_id
    s = row.streamer
    hist = user_history_train.get(u, [])
    if not hist:
        cf_scores.append(0.0)
        continue
    sims = [item_sim.get((s, h), 0.0) for h in set(hist)]
    cf_scores.append(np.mean(sims) if sims else 0.0)

cf_scores = np.array(cf_scores)

## Hybrid model

We combine logistic regression and collaborative filtering via weighted interpolation.

We do this because the two models capture complementary signals: LR uses behavioral features while CF uses similarity structure.

In [ ]:
alphas = np.linspace(0, 1, 21)
best_alpha = 0.5
best_alpha_auc = -1

for a in alphas:
    hyb = a * logit_scores + (1 - a) * cf_scores
    auc = roc_auc_score(y_test, hyb)
    if auc > best_alpha_auc:
        best_alpha_auc = auc
        best_alpha = a

hyb_scores = best_alpha * logit_scores + (1 - best_alpha) * cf_scores

## Threshold tuning

We select the probability threshold that maximizes F1 score.

We do this because optimal thresholds vary depending on metric choice.

In [ ]:
thresholds = np.linspace(0.01, 0.5, 50)
best_f1 = -1
best_th = 0.5

for th in thresholds:
    metrics = evaluate_basic(y_test, logit_scores, th=th)
    if metrics["f1"] > best_f1:
        best_f1 = metrics["f1"]
        best_th = th

## Final evaluation

We compare logistic regression, collaborative filtering, and the hybrid model using:
- accuracy
- ROC AUC
- precision
- recall
- F1
- precision@K
- recall@K

In [ ]:
print("LogReg:", evaluate_basic(y_test, logit_scores, th=best_th))
print("LogReg@K:", evaluate_at_k(test_df, logit_scores, k=TOP_K))

print("CF:", evaluate_basic(y_test, cf_scores, th=0.5))
print("CF@K:", evaluate_at_k(test_df, cf_scores, k=TOP_K))

print("Hybrid:", evaluate_basic(y_test, hyb_scores, th=best_th))
print("Hybrid@K:", evaluate_at_k(test_df, hyb_scores, k=TOP_K))

## Diagnostic plots

We visualize ROC, precision–recall curves, score distributions, and feature importances to understand model behavior.

In [ ]:
fpr_lr, tpr_lr, _ = roc_curve(y_test, logit_scores)
fpr_h, tpr_h, _ = roc_curve(y_test, hyb_scores)

plt.figure()
plt.plot(fpr_lr, tpr_lr)
plt.plot(fpr_h, tpr_h)
plt.plot([0,1],[0,1],"--")
plt.show()

prec_lr, rec_lr, _ = precision_recall_curve(y_test, logit_scores)
prec_h, rec_h, _ = precision_recall_curve(y_test, hyb_scores)

plt.figure()
plt.plot(rec_lr, prec_lr)
plt.plot(rec_h, prec_h)
plt.show()

plt.figure()
plt.hist(logit_scores[y_test == 0], bins=50, alpha=0.5)
plt.hist(logit_scores[y_test == 1], bins=50, alpha=0.5)
plt.show()

coef = logreg.coef_[0]
feat_importance = pd.Series(coef, index=feature_cols).sort_values(key=lambda x: abs(x), ascending=False)
feat_importance.head(15)

## Failure case analysis

We inspect top false positives and false negatives to understand where the model struggles.  
This helps identify systematic issues not visible from metrics alone.

In [ ]:
eval_df = test_df.copy()
eval_df["lr_score"] = logit_scores

false_positives = eval_df[eval_df["label"] == 0].sort_values("lr_score", ascending=False).head(10)
false_negatives = eval_df[eval_df["label"] == 1].sort_values("lr_score", ascending=True).head(10)

display(false_positives[["user_id","streamer","time_start","label","lr_score"]])
display(false_negatives[["user_id","streamer","time_start","label","lr_score"]])

## Save final model artifacts

We save the trained logistic regression model, scaler, feature list, and hybrid mixing parameter.

We do this because downstream inference requires consistent preprocessing and model weights.

In [ ]:
artifacts = {
    "logreg": logreg,
    "scaler": scaler,
    "feature_cols": feature_cols,
    "best_alpha": best_alpha,
    "best_threshold": best_th
}

with open("part2.pkl", "wb") as f:
    pickle.dump(artifacts, f)

Part 2 All Done